In [2]:
import json, csv, string, datetime
from dateutil.parser import parse
import pandas as pd

### Visualizando os dados

In [3]:
data = pd.read_json('DATASET.json',  encoding='utf-8', lines=True)
data.head()

,data,geo,id,nome,text
0,Sun May 06 20:41:33 +0000 2018,"[19.59433331, -155.97172783]",2.379071e+09,tiedyehobo,"majic sands beach, kona hawaiiawesome tunes by..."
1,Sun May 06 20:33:41 +0000 2018,"[19.56034, -154.9866]",2.929292e+07,queeraspoetry,Good am! 🤙🌋 @ Center for Spiritual Living in H...
2,Sun May 06 20:30:04 +0000 2018,"[19.0781667, -155.1876667]",8.149151e+17,Hazards_Network,Monday-January-2018::15:14: An earthquake of m...
3,Sun May 06 20:09:43 +0000 2018,"[20.02027778, -155.66777778]",4.142280e+08,NaomiCooper808,"#vacay #Ladat @ Waimea, Hawaii County, Hawai"
4,Sun May 06 19:46:08 +0000 2018,"[19.65, -155.994]",2.011251e+07,vioart527,"Last sunset in Hawaii . . . until next time,"


### Algumas informações sobre o dataset

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6023 entries, 0 to 6022
Data columns (total 5 columns):
data    6023 non-null object
geo     6023 non-null object
id      6023 non-null float64
nome    6023 non-null object
text    6023 non-null object
dtypes: float64(1), object(4)
memory usage: 235.4+ KB


### Campos nulos ?

In [6]:
data.isnull().sum()

data    0
geo     0
id      0
nome    0
text    0
dtype: int64

### Formatando datas

In [7]:
def parse_datetime(value):
    time_tuple = parsedate_tz(value)
    timestamp  = mktime_tz(time_tuple)
    return datetime.datetime.fromtimestamp(timestamp)

times = { "data": [], "time": [], "lat":  [], "long": [] }
for x, k in zip(data['data'].apply(parse), data['geo']):
    time_out = str(x).split(" ")
    geo_out  = k.replace("[","").replace("]","").split(',')
    times["data"].append(time_out[0])
    times["time"].append(str(time_out[1]).replace("+00:00",""))
    times["lat"].append(geo_out[0])
    times["long"].append(geo_out[1])
    
#Atualizações no dataframe
data['data'] = pd.Series(times['data'])
data['time'] = pd.Series(times['time'])
data['lat']  = pd.Series(times['lat'])
data['long'] = pd.Series(times['long'])
data.pop('geo')
data.columns = ['date', 'id_str','user_name','text', 'time', 'latitude', 'longitude']

### Verificando novamente valores nulos

In [8]:
data.isnull().sum()

date         0
id_str       0
user_name    0
text         0
time         0
latitude     0
longitude    0
dtype: int64

### Visualizando o dataframe

In [9]:
data.head()

,date,id_str,user_name,text,time,latitude,longitude
0,2018-05-06,2.379071e+09,tiedyehobo,"majic sands beach, kona hawaiiawesome tunes by...",20:41:33,19.59433331,-155.97172783
1,2018-05-06,2.929292e+07,queeraspoetry,Good am! 🤙🌋 @ Center for Spiritual Living in H...,20:33:41,19.56034,-154.9866
2,2018-05-06,8.149151e+17,Hazards_Network,Monday-January-2018::15:14: An earthquake of m...,20:30:04,19.0781667,-155.1876667
3,2018-05-06,4.142280e+08,NaomiCooper808,"#vacay #Ladat @ Waimea, Hawaii County, Hawai",20:09:43,20.02027778,-155.66777778
4,2018-05-06,2.011251e+07,vioart527,"Last sunset in Hawaii . . . until next time,",19:46:08,19.65,-155.994


# Trabalhando mais profundo > *removendo spam*

In [10]:
#ordenando pelo texto
data.sort_values('text', inplace = True)

In [11]:
#removendo valores nulos
data.dropna(inplace = True)

In [13]:
#mostando perfis de spam
#similaridade dos spams/bots ...  
#Mesmo horário, mesmo conteúdo da msg, mesma lat e long
spams = data[data['text'].duplicated( keep = False)]
spams

,date,id_str,user_name,text,time,latitude,longitude
1665,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
632,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
1319,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
1320,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
288,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
1664,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
5854,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
633,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
975,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358
974,2018-04-30,2.379071e+09,tiedyehobo,""" ganja"" the tiedye apprenticetQ1LaRy9E7#tiedy...",21:43:05,19.52575,-155.92358


### Removendo perfis de spam

In [14]:
#pegando os 'user_name' do perfis spams
for names in set(spams['user_name']):
    print(names)
    #Essa linha exclui todos os perfis com base no indice de cada 'user_name'
    data.drop(data[data['user_name'] == names ].index, inplace = True)

# teste de perfil deletado
#data[data['user_name'].str.match('QuakesToday')]

MarkOwnsUrFace
sstnblcstlns
clarklittle
TiffanyMCarlson
MarkNakashima
paulgotelweb
Aceman808
Travelmonkie
arielkcurry
BigEarthquakes
Verticalviking
welshietouring
KonaHomeTeam
USGSBigQuakes
drummerchris
USGSted
Astro_Herv
shellokitty78
PattyKWilliams
Chase_Schnur
DeVante1
THEREALPUNK
OhanaSuh1
eq_map_es2
qball13z
mikeygomikey
radxnickelx
thesnorkelstore
FourSeasonsJobs
LahainaGallery
chazatronic
symbolicimagery
sdbrady
sunflwer1975
u_nagi2356
DougTho89360871
AngelaConners_
Rabuuf
pintsizegourmet
AAgbayani
TrevorHughes
eq_map_es
tmj_hi_nursing
foodoirs
5oclockbot
QuakesToday
Indica_Honey
NotableBirdsHI
paderan
vinillaskyz
ynoelani
megantg
LastQuake
disasteraware
miu_north
itsnotben
GeorgeIzvernari
chrisgrohusko
fafjrock
patogawa
RobertPerryHI
tiedyehobo
cyulit
glemaud
eq_map_ww
tmj_usa_aero
ShortiniPhotog
_tamikaanne
vioart527
hellocamille02
PineappleClan
syvideo
HiloHennaHut
earthshook
jammaquino
NaomiCooper808
ChiokeMusic
MauiDarci
WordsOfVikram
angieinpixels
KingsShopsHI
alikaboy4mus

### Mostrando grupos de tweets por usuário

In [15]:
data.sort_values('user_name', inplace = True)
data.head()

,date,id_str,user_name,text,time,latitude,longitude
2406,2018-05-09,2.992176e+08,0wwl,"Island living 🏝🥥🌺 @ Kapalua, Hawai",03:44:09,20.9967,-156.653
5042,2018-05-03,1.291239e+09,1fsmith1,Blessed to be home in Hawai’i for a few weeks!...,01:25:05,19.6346908,-155.9898393
2594,2018-05-08,1.209477e+08,1hawaiiboy,The tools at the ready for afternoon strawberr...,20:13:31,19.723,-156.005
5837,2018-04-30,1.209477e+08,1hawaiiboy,Spirits @ Sansei Waikolo,23:31:26,19.91383842,-155.87940562
5281,2018-05-02,1.209477e+08,1hawaiiboy,Pint and a shot#guinness #tullamoredew @,09:01:52,19.92545,-155.88433


### Exportando Json -> DataFrame -> CSV | *E testando*

In [16]:
data.to_csv("out_dataset_no_bots.csv",  index = None)

In [17]:
pd.read_csv("out_dataset_no_bots.csv").head()

,date,id_str,user_name,text,time,latitude,longitude
0,2018-05-09,2.992176e+08,0wwl,"Island living 🏝🥥🌺 @ Kapalua, Hawai",03:44:09,20.996700,-156.653000
1,2018-05-03,1.291239e+09,1fsmith1,Blessed to be home in Hawai’i for a few weeks!...,01:25:05,19.634691,-155.989839
2,2018-05-08,1.209477e+08,1hawaiiboy,The tools at the ready for afternoon strawberr...,20:13:31,19.723000,-156.005000
3,2018-04-30,1.209477e+08,1hawaiiboy,Spirits @ Sansei Waikolo,23:31:26,19.913838,-155.879406
4,2018-05-02,1.209477e+08,1hawaiiboy,Pint and a shot#guinness #tullamoredew @,09:01:52,19.925450,-155.884330
